# Generating the serving layer

In this exercise, we'll use Spark structured streaming to generate the serving layer. Specifically, we'll write each event to a single table in InfluxDB. We'll use [the `influxdb` Python library](https://influxdb-python.readthedocs.io/en/latest/api-documentation.html) since there is no dedicated InfluxDB Spark connector.

In [1]:
%%bash
# Install the required Python 3 dependencies
python3 -m pip install kafka-python influxdb  # type: ignore

First, create the `process` function that writes a single row to InfluxDB. Add the `visitor_browser` and `visitor_country` as tags, `ts_ingest` as the time and the other values as `fields`.

In [ ]:
from influxdb import InfluxDBClient

client = InfluxDBClient('localhost', 8086, 'root', 'root', 'example')
client.create_database('data')

def process(row):
    client.write_points([{
            "measurement": "clicks",
            "tags": {
                "visitor_browser": row['visitor_browser'],
                "visitor_country": row['visitor_country'],
            },
            "time": row['ts_ingest'],
            "fields": {
                "visitor_platform": row['visitor_platform'],
                "article_title": row['article_title'],
                "article": row['article'],
                "visitor_os": row['visitor_os'],
                "visitor_page_timer": row['visitor_page_timer'],
                "visitor_page_height": row['visitor_page_height'],
            },
        }])


To query the database we can send a HTTP request to the db. More info can be found in the InfluxDB [documentation](https://docs.influxdata.com/influxdb/v1.7/guides/querying_data/). Use the local terminal for curl commands since the notebook image does not have it installed.

```text
curl -G 'http://localhost:8086/query?pretty=true' --data-urlencode "db=data" --data-urlencode "q=SELECT * FROM \"clicks\""
```

Create a Spark context and specify that the python spark-kafka libraries need to be added.

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.3 pyspark-shell'

import pyspark 
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

sc = SparkContext()
sc.setLogLevel("WARN")
spark = SparkSession(sc)

Create a streaming DataFrame that represents the events received from the Kafka topic `clicks-cleaned`.

In [ ]:
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers","localhost:9092") \
    .option("subscribe", "clicks-cleaned") \
    .option("startingOffsets", "latest") \
    .load()

Cast the json to columns in the DataFrame. Make sure to use TimestampType for the `ts_ingest` since we already converted it in the `clean` notebook.

In [ ]:
schema = StructType([
    StructField("visitor_platform", StringType()),
    StructField("ts_ingest", TimestampType()),
    StructField("article_title", StringType()),
    StructField("visitor_country", StringType()),
    StructField("visitor_os", StringType()),
    StructField("article", StringType()),
    StructField("visitor_browser", StringType()),
    StructField("visitor_page_timer", IntegerType()),
    StructField("visitor_page_height", IntegerType()),
])

dfs = df.selectExpr("CAST(value AS STRING)") \
      .select(from_json(col("value"), schema) \
      .alias("clicks"))

df_data = dfs.select("clicks.*")

Use a Spark `foreach` statement to call the `process` function for each row and start the query [docs](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#foreach).

In [ ]:
query = df_data.writeStream.foreach(process).start()
query.awaitTermination()

# Debug in terminal
# query = df_data_grouped.writeStream.outputMode("update").option("truncate", "false").format("console").start()
# query.awaitTermination()